# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



**Motivation**


### Fake News Classification Dataset Information

The [Fake News Classification Dataset](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification) contains real and fake news articles.

- **Index:**: Index.
- **title:** Title of news article.
- **text:** Text content of news article.
- **label:** Whether news article is real (1) or fake (0).

The Fake News Dataset is split into 3 `csv` files (`part1.csv`, `part2.csv`, `part3.csv`) so that size does not exceed size limit to push changes to GitHub.

## Import Libraries

In [106]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Text Preprocessing and NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud


## Data Preparation (Loading CSV)

Load the 3 Fake News `csv` files into pandas DataFrames
- `part1.csv` is loaded into `data_raw1` DataFrame.
- `part2.csv` is loaded into `data_raw2` DataFrame.
- `part3.csv` is loaded into`data_raw3` DataFrame.

In [107]:
data_raw1 = pd.read_csv('part1.csv')
data_raw2 = pd.read_csv('part2.csv')
data_raw3 = pd.read_csv('part3.csv')

In [108]:
data_raw1.info()
print("Dataframe 1 Shape: ", data_raw1.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24044 non-null  int64 
 1   title       23894 non-null  object
 2   text        24033 non-null  object
 3   label       24044 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.5+ KB
Dataframe 1 Shape:  (24044, 4)


In [109]:
data_raw2.info()
print("Dataframe 2 Shape: ", data_raw2.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24044 non-null  int64 
 1   title       23849 non-null  object
 2   text        24030 non-null  object
 3   label       24044 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.5+ KB
Dataframe 2 Shape:  (24044, 4)


In [110]:
data_raw3.info()
print("Dataframe 3 Shape: ", data_raw3.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24046 entries, 0 to 24045
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24046 non-null  int64 
 1   title       23833 non-null  object
 2   text        24032 non-null  object
 3   label       24046 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.6+ KB
Dataframe 3 Shape:  (24046, 4)


In [111]:
data_raw1.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [112]:
data_raw2.head()

,Unnamed: 0,title,text,label
0,24044,Massachusetts governor would sign transgender ...,BOSTON (Reuters) - Massachusetts Governor Char...,0
1,24045,"Weakened Militarily, ISIS Still Has Power to S...","In the past few weeks, the Islamic State has s...",0
2,24046,SARAH JESSICA PARKER FEARS She’ll Be Attacked ...,"With absolutely NOTHING to back it up, Sarah J...",1
3,24047,Rosenstein may need to recuse himself from Rus...,WASHINGTON (Reuters) - U.S. Deputy Attorney Ge...,0
4,24048,Cop Arrested for Using Police Status to Rape L...,Home / Badge Abuse / Cop Arrested for Using Po...,1


In [113]:
data_raw3.head()

,Unnamed: 0,title,text,label
0,48088,Blue State Blues: The White House Loves the Ru...,The Trump White House seems quietly to be enjo...,0
1,48089,"Harassment Crisis Builds at Fox News, Despite ...",When the anchor Gretchen Carlson filed a bombs...,0
2,48090,Former Ku Klux Klan leader Duke runs for U.S. ...,"BATON ROUGE, La. (Reuters) - David Duke, a for...",0
3,48091,NYTIMES: Sessions’ and Bannon’s Department of ...,Emily Bazelon writes in the New York Times on ...,0
4,48092,UPDATE: WIKILEAKS CLINTON Document Dump Back On!,Hillary Clinton strategist Bob Beckel called f...,1


## Combine all 3 dataframes into 1
Here we combine all 3 dataframes (`data_raw1`, `data_raw2`, `data_raw3`) into 1 dataframe (`data_raw`) by concatenating along rows.

We also reset the `index` and drops the old `index` column

In [114]:
# Combining all 3 dataframes into 1
data_raw = pd.concat([data_raw1, data_raw2, data_raw3], axis=0)

# Reset index and drop old index column
data_raw = data_raw.reset_index(drop=True)

data_raw.info()
print("Dataframe Shape: ", data_raw.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB
Dataframe Shape:  (72134, 4)


In [115]:
data_raw

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


## Feature Selection
Here we select the relevant features for fake news classification
- `title`, `text`, `label`
- Create a new DataFrame (`data`) by selecting the specifc columns mentioned above from the original DataFrame `data_raw`.

### Remove Duplicate Rows
- Drop duplicate rows from the dataframe (`data`).

In [116]:
data = data_raw[['title', 'text', 'label',]]
print(type(data))
print(data.head())

# Shape before dropping duplicates
print("The old shape is: ", data.shape)

data = data.drop_duplicates()

# Display the new dataframe shape
print("The new shape is: ", data.shape)

<class 'pandas.core.frame.DataFrame'>
                                               title  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1                                                NaN   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3  Bobby Jindal, raised Hindu, uses story of Chri...   
4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  
The old shape is:  (72134, 3)
The new shape is:  (63678, 3)


## Remove Outliers

### `text`

The `text` column of `data`, which is of string type, may contain values with unusually long lengths, indicating the presence of outliers. We will identify the outliers using [Z-score method].

1. Create a new column `text_length` in the DataFrame `data` by calculating the length of each review. (Set the value as 0 if the correponding `text` column has NaN values.)
2. Check the statistics of `text_length` using `describe()` method.
3. Calculate the mean and standard deviation of the `text_length` column.
4. Set the Z-score threshold for identifying outliers to 3.
5. Identify outliers of the `text_length` column and set the corresponding `text` to np.nan.
6. Drop the `text_length` column from the DataFrame.

In [117]:
data['text_length'] = data['text'].apply(lambda x: len(x) if pd.notna(x) else 0)
print(data.head(3))

TL = data["text_length"]
stats_TL = TL.describe()
print(stats_TL)

                                               title  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1                                                NaN   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   

                                                text  label  text_length  
0  No comment is expected from Barack Obama Membe...      1         5049  
1     Did they post their votes for Hillary already?      1           46  
2   Now, most of the demonstrators gathered last ...      1          216  
count     63678.000000
mean       3276.752866
std        3636.269585
min           0.000000
25%        1428.000000
50%        2456.000000
75%        4095.000000
max      142961.000000
Name: text_length, dtype: float64


In [118]:
mean_TL = TL.mean()
# print(mean_TL)

sd_TL = TL.std()
# print(sd_TL)

threshold = 3

z_score = zscore(TL)
# print(z_score)

# Remove 'text' of lengths that are greater than 3 standard deviations above the mean
data.loc[abs(z_score) > threshold, 'text'] = np.nan
# print(data.head(3))

data = data.drop("text_length", axis=1)
# print(data.head(3))

### `title`

Similarly, the `title` column of `data` (of type `str`) may also contain values with unusually long lengths, indicating the presence of outliers.

1. Create a new column `title_length` in the DataFrame `data` by calculating the length of each price value. (Set the value as 0 if the correponding `title` column has NaN values.)
2. Check the statistics of `title_length` using `describe()` method and display its unique values.
3. Identify the outlier values by inspecting the content in `title` corresponding to the abnormal value in `title_length` and set the corresponding value of `title` to np.nan.
4. Drop the `title_length` column from the DataFrame.

In [119]:
data['title_length'] = data['title'].apply(lambda x: len(x) if pd.notna(x) else 0)
print(data.head(3))

TL = data["title_length"]
stats_TL = TL.describe()
print(stats_TL)

                                               title  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1                                                NaN   
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   

                                                text  label  title_length  
0  No comment is expected from Barack Obama Membe...      1           130  
1     Did they post their votes for Hillary already?      1             0  
2   Now, most of the demonstrators gathered last ...      1           137  
count    63678.000000
mean        74.568344
std         23.619850
min          0.000000
25%         61.000000
50%         72.000000
75%         86.000000
max        456.000000
Name: title_length, dtype: float64


In [120]:
mean_TL = TL.mean()
# print(mean_TL)

sd_TL = TL.std()
# print(sd_TL)

threshold = 3

z_score = zscore(TL)
# print(z_score)

# Remove 'title' of lengths that are greater than 3 standard deviations above the mean
data.loc[abs(z_score) > threshold, 'title'] = np.nan
# print(data.head(3))

data = data.drop("title_length", axis=1)
# print(data.head(3))

In [121]:
data.isnull().sum()

title    1344
text      877
label       0
dtype: int64

## Feature Engineering

### Create new column `full_review`
Since there are some rows with empty `text` and `title`, we will concatenate both columns (`text` and `title`) to form a new column `full_review`.
1. Replace `NaN` values in `text` and `title` with an empty string

2. Combine `text` and `title` into `full_review`

3. Strip any leading/trailing whitespaces in `full_review`

4. Drop `text` and `title` columns

In [122]:
# 1) Fill NaN values in 'text' and 'title' with an empty string
data['title'] = data['title'].fillna('')
data['text'] = data['text'].fillna('')

# 2) Combine 'text' and 'title' into 'full_review'
data['full_review'] = data['text'] + " " + data['title']

# 3) Strip any leading/trailing whitespace
data['full_review'] = data['full_review'].str.strip()

# 4) Drop `text` and `title` columns
data = data.drop(columns = ['text', 'title'])

# Check if the 'full_review' column was added and if 'text' and 'title' columns has been dropped
data.head()

,label,full_review
0,1,No comment is expected from Barack Obama Membe...
1,1,Did they post their votes for Hillary already?
2,1,"Now, most of the demonstrators gathered last n..."
3,0,A dozen politically active pastors came here f...
4,1,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
